In [ ]:
import gc
import pickle as pkl
import numpy as np
from numba import njit

---
---
---

In [ ]:
offsets = np.array([[-1, -1, -1], [-1, -1,  0], [-1, -1,  1],
                    [-1,  0, -1], [-1,  0,  0], [-1,  0,  1], 
                    [-1,  1, -1], [-1,  1,  0], [-1,  1,  1],
                    [ 0, -1, -1], [ 0, -1,  0], [ 0, -1,  1],
                    [ 0,  0, -1],               [ 0,  0,  1],
                    [ 0,  1, -1], [ 0,  1,  0], [ 0,  1,  1],
                    [ 1, -1, -1], [ 1, -1,  0], [ 1, -1,  1],
                    [ 1,  0, -1], [ 1,  0,  0], [ 1,  0,  1],
                    [ 1,  1, -1], [ 1,  1,  0], [ 1,  1,  1]])

In [ ]:
def shift_cube(data_cube, shift):
    
    
    
    shifted_cube = np.empty_like(data_cube)
    x_shift, y_shift, z_shift = shift
    
    
    
    shifted_cube[max(0, x_shift) :min(data_cube.shape[0], data_cube.shape[0] + x_shift),
                 max(0, y_shift) :min(data_cube.shape[1], data_cube.shape[1] + y_shift),
                 max(0, z_shift) :min(data_cube.shape[2], data_cube.shape[2] + z_shift)] = \
    data_cube[   max(0, -x_shift):min(data_cube.shape[0], data_cube.shape[0] - x_shift),
                 max(0, -y_shift):min(data_cube.shape[1], data_cube.shape[1] - y_shift),
                 max(0, -z_shift):min(data_cube.shape[2], data_cube.shape[2] - z_shift)]
    
    
    
    # Reflect along x-axis
    if x_shift != 0:
        if x_shift == 1: vals_x = [0,-1]
        else:            vals_x = [-1,0]
        shifted_cube[vals_x[0],
                     max(0, y_shift) :min(data_cube.shape[1], data_cube.shape[1] + y_shift),
                     max(0, z_shift) :min(data_cube.shape[2], data_cube.shape[2] + z_shift)] = \
        data_cube[   vals_x[1],
                     max(0, -y_shift):min(data_cube.shape[1], data_cube.shape[1] - y_shift),
                     max(0, -z_shift):min(data_cube.shape[2], data_cube.shape[2] - z_shift)]

    if y_shift != 0:
        if y_shift == 1: vals_y = [0,-1]
        else:            vals_y = [-1,0]
        shifted_cube[max(0, x_shift) :min(data_cube.shape[0], data_cube.shape[0] + x_shift),
                     vals_y[0],
                     max(0, z_shift) :min(data_cube.shape[2], data_cube.shape[2] + z_shift)] = \
        data_cube[   max(0, -x_shift):min(data_cube.shape[0], data_cube.shape[0] - x_shift),
                     vals_y[1],
                     max(0, -z_shift):min(data_cube.shape[2], data_cube.shape[2] - z_shift)]

    if z_shift != 0:
        if z_shift == 1: vals_z = [0,-1]
        else:            vals_z = [-1,0]
        shifted_cube[max(0, x_shift) :min(data_cube.shape[0], data_cube.shape[0] + x_shift),
                     max(0, y_shift) :min(data_cube.shape[1], data_cube.shape[1] + y_shift),
                     vals_z[0],] = \
        data_cube[   max(0, -x_shift):min(data_cube.shape[0], data_cube.shape[0] - x_shift),
                     max(0, -y_shift):min(data_cube.shape[1], data_cube.shape[1] - y_shift), 
                     vals_z[1]]
    
    
    
    if x_shift != 0 and y_shift != 0:
        if x_shift == 1: vals_x = [0,-1]
        else:            vals_x = [-1,0]
        if y_shift == 1: vals_y = [0,-1]
        else:            vals_y = [-1,0]
        
        shifted_cube[vals_x[0],
                     vals_y[0],
                     max(0, z_shift) :min(data_cube.shape[2], data_cube.shape[2] + z_shift)] = \
        data_cube[   vals_x[-1],
                     vals_y[-1],
                     max(0, -z_shift):min(data_cube.shape[2], data_cube.shape[2] - z_shift)]
    
    if x_shift != 0 and z_shift != 0:
        if x_shift == 1: vals_x = [0,-1]
        else:            vals_x = [-1,0]
        if z_shift == 1: vals_z = [0,-1]
        else:            vals_z = [-1,0]
        shifted_cube[vals_x[0],
                     max(0, y_shift) :min(data_cube.shape[1], data_cube.shape[1] + y_shift),
                     vals_z[0]] = \
        data_cube[   vals_x[-1],
                     max(0, -y_shift):min(data_cube.shape[1], data_cube.shape[1] - y_shift),
                     vals_z[-1]]
    
    if y_shift != 0 and z_shift != 0:
        if y_shift == 1: vals_y = [0,-1]
        else:            vals_y = [-1,0]
        if z_shift == 1: vals_z = [0,-1]
        else:            vals_z = [-1,0]
        shifted_cube[max(0, x_shift) :min(data_cube.shape[0], data_cube.shape[0] + x_shift),
                     vals_y[0],
                     vals_z[0]] = \
        data_cube[   max(0, -x_shift):min(data_cube.shape[0], data_cube.shape[0] - x_shift),
                     vals_y[-1],
                     vals_z[-1]]
    
    
    
    if x_shift != 0 and y_shift != 0 and z_shift != 0:
        if x_shift == 1: vals_x = [0,-1]
        else:            vals_x = [-1,0]
        if y_shift == 1: vals_y = [0,-1]
        else:            vals_y = [-1,0]
        if z_shift == 1: vals_z = [0,-1]
        else:            vals_z = [-1,0]
        shifted_cube[vals_x[0],
                     vals_y[0],
                     vals_z[0]] = \
        data_cube[   vals_x[-1],
                     vals_y[-1],
                     vals_z[-1]]
    
    
    
    return shifted_cube

---

In [ ]:
@njit
def condition_neighbor(ll_i, ll_i_00, size):
    
    
    ll_i_diff = np.abs(ll_i - ll_i_00)

    condition = ((ll_i_diff[:, 0] <= 1) | (ll_i_diff[:, 0] == size-1)) & ((ll_i_diff[:, 1] <= 1) | (ll_i_diff[:, 1] == size-1)) & ((ll_i_diff[:, 2] <= 1) | (ll_i_diff[:, 2] == size-1))
    
    all_neighbors = [list(i) for i in ll_i[condition]]
    all_indices   = np.array([i for i in range(len(ll_i))])[condition]
    
    return all_neighbors, all_indices

In [ ]:
@njit
def pairs_finder_1(ll_i, size):
    
    
    ll_i  = [list(i) for i in ll_i]
    pairs = [[[0,0,0]]][:0]
     
    while len(ll_i) != 0:
        
        # Form a new pair
        ll_i_00 = ll_i[0]; ll_i = ll_i[1:]
        pairs.append([ll_i_00])
        
        # get its neighbors from ll_i (excluding itself) and their index numbers
        all_neighbors, all_indices = condition_neighbor(np.array(ll_i), np.array(ll_i_00), size)
        
        for neighbor in all_neighbors: pairs[-1].append(neighbor)
        
        # Remove these cells from the level.
        inn = 0
        for index in all_indices:
            i1 = index-inn; inn += 1
            ll_i = ll_i[:i1] + ll_i[i1+1:]
            
        
        old_neighbors_cells = all_neighbors.copy()

        found_some = True   # We must've... we're only looking at pairs here.
        while found_some:
            found_some = False

            new_neighbors_cells = [[0,0,0]][:0]

            for nc_0 in old_neighbors_cells:

                if len(ll_i) != 0:
                    
                    # get its neighbors from ll_i (excluding itself) and their index numbers
                    all_neighbors, all_indices = condition_neighbor(np.array(ll_i), np.array(nc_0), size)
    
                    
                    for neighbor in all_neighbors:
                        found_some = True
                        pairs[-1].append(neighbor)
                        new_neighbors_cells.append(neighbor)
    
                    inn = 0
                    for index in all_indices:
                        i1 = index-inn; inn += 1
                        ll_i = ll_i[:i1] + ll_i[i1+1:]
                
                old_neighbors_cells = new_neighbors_cells.copy()
        
        
    return pairs

---
---
---